In [1]:
import sys, time
import torch
sys.path.insert(0, '../../Models')
sys.path.insert(0, '../../Utils')
from _utils import  sample_random_glue_stsb, collect_info_for_metric, save_info, \
                    get_continuation_mapping, get_continuous_attributions, get_continuous_raw_inputs,\
                    attr_normalizing_func
from preload_models import get_stsb_tokenizer_n_model

In [2]:
stsb_data_raw, targets, idxs = sample_random_glue_stsb()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

[ 436  138  931 1462 1491  577  873  899  268  603 1341  648  941  901
  335 1306  234  132 1257 1433 1351 1484  977  146 1275  225  396  757
  949 1151  605 1041  350  788  626   88  599 1080  882  927 1209  647
  486  214  317  895  897   50 1054 1454]


In [3]:
tokenizer, model = get_stsb_tokenizer_n_model()

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import Lime
from captum._utils.models.linear_model import SkLearnLasso
from captum.attr import visualization 

In [6]:
lime = Lime(model, interpretable_model=SkLearnLasso(alpha=0.0003))

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0)
    detokenized = [t.replace('Ġ', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()
    print(f'pred_prob {pred_prob*5}')

     #categorizing results
    pred_class = 'Similar' if pred_prob > 0.5 else 'Not Similar' 
    true_class = 'Similar' if target > 2.5 else 'Not Similar' 
    
    #attribution algorithm working
    attri_start_time = time.time()
    attribution = lime.attribute(input_emb, n_samples=3000, show_progress=True, perturbations_per_eval=1000)
    print(f'attribution took {time.time() - attri_start_time:2f} seconds')
    
    attribution[torch.isnan(attribution)] = 0
    word_attributions = attribution.squeeze(0).sum(dim=1)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'Similar' if attr_score > 0.5 else 'Not Similar'
    convergence_score = None
    
    if torch.all(torch.isnan(attr_score)):
        print(f'nan attri score')
        word_attributions = torch.zeros(word_attributions.size())
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
#     print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
#   collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(stsb_data_raw, targets), start=1):
#     example_1 = "Here are many samples: Pbase.com Out of the 16k photos there, I'm sure some are with a d700."
#     example_2 = "It's a 1:1 lens, so that means that the size of the subject will be the same size on the sensor."
#     datum_raw, target = [example_1, example_2],  1
    print(f'i: {i-1}')
    print(f'Raw review: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record]))   


i: 0
Raw review: ['Two dogs are sniffing something in the rocks.', 'Two dogs running down a path in the woods.']
GT target: 1.600000023841858


/home/user/anaconda3/envs/py38/lib/python3.9/site-packages/captum/attr/_core/lime.py:1088: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(


pred_prob 2.0290477573871613


Lime attribution: 100%|███████████████████████████| 3/3 [01:51<00:00, 37.01s/it]


attribution took 117.434270 seconds
before zeroing tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.0192e-04,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [-8.6970e-05, -9.8200e-04,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -2.4136e-04,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]])
after zeroing tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.0192e-04,  ...,  0.0000e+00,
           

...

<IPython.core.display.HTML object>
i: 49
Raw review: ["Foreign companies eye new 'opening-up'", 'More carriers likely in China']
GT target: 0.6000000238418579
pred_prob 0.07349737454205751


Lime attribution: 100%|███████████████████████████| 3/3 [00:49<00:00, 16.36s/it]


attribution took 52.763262 seconds
before zeroing tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
after zeroing tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
nan attri score
word attr tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
conti attr [tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.)]
detokenized ['<s>', 'Foreign', 'companies', 'eye', 'new', "'", 'opening', '-', 'up', "'", '</s>', '</s>', 'More', 'carriers',

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Not Similar,Not Similar (0.01),Not Similar,nan,#s Foreign companies eye new 'opening-up' #/s #/s More carriers likely in China #/s


<IPython.core.display.HTML object>


In [9]:
save_info(idxs, stsb_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, fname='lime_out.pkl')

{'indices': [436,
  138,
  931,
  1462,
  1491,
  577,
  873,
  899,
  268,
  603,
  1341,
  648,
  941,
  901,
  335,
  1306,
  234,
  132,
  1257,
  1433,
  1351,
  1484,
  977,
  146,
  1275,
  225,
  396,
  757,
  949,
  1151,
  605,
  1041,
  350,
  788,
  626,
  88,
  599,
  1080,
  882,
  927,
  1209,
  647,
  486,
  214,
  317,
  895,
  897,
  50,
  1054,
  1454],
 'raw_data': [['Two dogs are sniffing something in the rocks.',
   'Two dogs running down a path in the woods.'],
  ['A woman is slicing a potato.', 'A woman is slicing carrot.'],
  ['It was before the season began because Amy saw the woman before the Silence took her.',
   "The exact timeline hasn't been listed yet, but there are a few assumptions we can make."],
  ['Protests for kidnapped girls banned in Nigerian capital',
   "Scores 'killed' in Boko Haram raid in Nigeria"],
  ['Oil falls in Asian trade', 'Oil prices down in Asian trade'],
  ['There are people out on the street.', 'People are out on the street.'],
 